In [1]:
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import numpy as np
from util.models import MODELS
from util.tasks import TASKS
from notebook_utils import *

%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()

from tqdm import tqdm
from distill.repsim_util import get_reps

In [2]:
task_name = 'word_sv_agreement_vp'
chkpt_dir='../tf_ckpts'
task = TASKS[task_name](get_task_params(), data_dir='../data')
cl_token = task.databuilder.sentence_encoder().encode(constants.bos)

Vocab len:  10032


In [3]:
config={'student_exp_name':'test',
    'teacher_exp_name':'test',
    'teacher_config':'small_lstm_v4',
    'task_name':'word_sv_agreement_vp',
    'student_model':'cl_lstm',
    'teacher_model':'cl_lstm',
    'student_config':'small_lstm_v4',
    'distill_config':'base',
    'distill_mode':'rep_online',
    'chkpt_dir':'../tf_ckpts',
       }

std_hparams=get_model_params(task, config['student_model'], config['student_config'])
std_hparams.output_attentions = True
std_hparams.output_embeddings = True
std_hparams.output_hidden_states = True

model1, _ = get_student_model(config, task, std_hparams, cl_token)

tchr_hparams=get_model_params(task, config['teacher_model'], config['teacher_config'])
tchr_hparams.output_attentions = True
tchr_hparams.output_embeddings = True
tchr_hparams.output_hidden_states = True

model2, _ = get_teacher_model(config, task, tchr_hparams, cl_token)


model config: small_lstm_v4
{'hidden_dim': 256, 'embedding_dim': 256, 'depth': 2, 'hidden_dropout_rate': 0.8, 'input_dropout_rate': 0.2, 'initializer_range': 0.1}
model config: small_lstm_v4
{'hidden_dim': 256, 'embedding_dim': 256, 'depth': 2, 'hidden_dropout_rate': 0.8, 'input_dropout_rate': 0.2, 'initializer_range': 0.1}
student_checkpoint: ../tf_ckpts/word_sv_agreement_vp/rep_online_base_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_test_student_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_test
No checkpoint found ../tf_ckpts/word_sv_agreement_vp/rep_online_base_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_test_student_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_test
model config: small_lstm_v4
{'hidden_dim': 256, 'embedding_dim': 256, 'depth': 2, 'hidden_dropout_rate': 0.8, 'input_dropout_rate': 0.2, 'initializer_range': 0.1}
No checkpoint found ../tf_ckpts/word_sv_agreement_vp/cl_lstm_em-256_h-256_d-2_hdr

In [13]:
print(model1.rep_layer)
print(model1.rep_index)
print(task.output_padding_symbol)


None
1
-1


In [11]:
for inputs, labels in task.valid_dataset:
    reps1 = get_reps(inputs, model1, index=model1.rep_index, layer=model1.rep_layer)
    reps2 = get_reps(inputs, model2, index=model2.rep_index, layer=model1.rep_layer)
    reploss = task.get_rep_loss()
    print(reploss(reps1, reps2, task.output_padding_symbol))

tf.Tensor(0.104028225, shape=(), dtype=float32)
tf.Tensor(0.10028511, shape=(), dtype=float32)
tf.Tensor(0.11930311, shape=(), dtype=float32)
tf.Tensor(0.11535907, shape=(), dtype=float32)


KeyboardInterrupt: 